In [22]:
import numpy as np
import torch
import torch.nn.functional as F
from time import time

In [18]:
# Define the dimensions of the array
rows = 3
cols = 3

# Create a 1D array of ascending integers
values = np.arange(1, rows * cols + 1)

# Reshape the 1D array into a 2D array
array_2d = values.reshape(rows, cols)

print(array_2d)

[[1 2 3]
 [4 5 6]
 [7 8 9]]


In [19]:
padded_array = np.pad(array_2d, 1, mode="wrap")
print(padded_array)

[[9 7 8 9 7]
 [3 1 2 3 1]
 [6 4 5 6 4]
 [9 7 8 9 7]
 [3 1 2 3 1]]


In [29]:
moore_neighborhood = np.array(
    [[1, 1, 1],
    [1, 0, 1],
    [1, 1, 1]]
)

neumann_neighborhood = np.array(
    [[0, 1, 0],
    [1, 0, 1],
    [0, 1, 0]]
)

In [30]:
def get_neighborhood_counts(universe, neighborhood):
    padded_universe = np.pad(universe, 1, mode="wrap")
    
    return F.conv2d(input=torch.tensor(padded_universe).unsqueeze(0).unsqueeze(0), 
                    weight=torch.tensor(neighborhood).unsqueeze(0).unsqueeze(0)).squeeze().numpy()


In [32]:
t0 = time()

result_tensor = F.conv2d(input=torch.tensor(padded_array).unsqueeze(0).unsqueeze(0), 
             weight=torch.tensor(moore_neighborhood).unsqueeze(0).unsqueeze(0), 
             bias=None,
             padding="valid").squeeze().numpy()

print(f"{time()-t0}")

0.003330230712890625


In [36]:
for i in range(256, 512):
    
    t0 = time()
    random_array = np.random.randint(2, size=(i, i))
    neighborhood_counts = get_neighborhood_counts(random_array, moore_neighborhood)
    
    print(f"size: {i} - dt: {time()-t0}")

size: 256 - dt: 0.00584721565246582
size: 257 - dt: 0.004482269287109375
size: 258 - dt: 0.0038607120513916016
size: 259 - dt: 0.003572702407836914
size: 260 - dt: 0.003998756408691406
size: 261 - dt: 0.0037255287170410156
size: 262 - dt: 0.0032989978790283203
size: 263 - dt: 0.003150463104248047
size: 264 - dt: 0.005403757095336914
size: 265 - dt: 0.005347728729248047
size: 266 - dt: 0.004183292388916016
size: 267 - dt: 0.0036919116973876953
size: 268 - dt: 0.00409388542175293
size: 269 - dt: 0.004170417785644531
size: 270 - dt: 0.0055959224700927734
size: 271 - dt: 0.00417017936706543
size: 272 - dt: 0.00522613525390625
size: 273 - dt: 0.00505518913269043
size: 274 - dt: 0.0063457489013671875
size: 275 - dt: 0.005019187927246094
size: 276 - dt: 0.003863811492919922
size: 277 - dt: 0.004693746566772461
size: 278 - dt: 0.0035953521728515625
size: 279 - dt: 0.004820823669433594
size: 280 - dt: 0.004389524459838867
size: 281 - dt: 0.005690097808837891
size: 282 - dt: 0.004219532012939453

In [39]:
def gol():
    total_time = 0.0
    for i in range(1000):
        t0 = time()
        universe = np.random.randint(2, size=(512, 512))
        next_generation = np.copy(universe)
        neighborhood_counts = get_neighborhood_counts(universe, moore_neighborhood)
        
        alive = np.where(universe == 1)
        dead = np.where(universe < 1)
        
        next_generation[alive] = np.where((neighborhood_counts[alive] == 2) | (neighborhood_counts[alive] == 3), 1.0, 0)
        next_generation[dead] = np.where(neighborhood_counts[dead] == 3, 1.0, 0)
        
        universe = next_generation
        
        time_spend = time() - t0
        total_time += time_spend
        # print(f"dt: {time_spend}")
        
    print(f"total_time: {total_time}")

In [40]:
gol()

total_time: 18.197903871536255
